# Custom String Evaluator

เราสามารถที่จะสร้างแบบประเมิน custom string evaluators ได้โดยใช้ StringEvaluators ตัวอย่างเช่นเราสามารถสร้างประเมินผ่านเมทริก perplexity โดยใช้ไลรารี่จาก huggingface 
เพื่อวัดข้อความที่ gen ออกมาดีแค่ไหน หรืออาจจะนำไปใช้กับ BELU score, ROUGH score ได้  

In [5]:
from typing import Any, Optional

from evaluate import load
from langchain.evaluation import StringEvaluator


class PerplexityEvaluator(StringEvaluator):
    """Evaluate the perplexity of a predicted string."""

    def __init__(self, model_id: str = "gpt2"):
        self.model_id = model_id
        self.metric_fn = load(
            "perplexity", module_type="metric", model_id=self.model_id, pad_token=0
        )

    def _evaluate_strings(
        self,
        *,
        prediction: str,
        reference: Optional[str] = None,
        input: Optional[str] = None,
        **kwargs: Any,
    ) -> dict:
        results = self.metric_fn.compute(
            predictions=[prediction], model_id=self.model_id
        )
        ppl = results["perplexities"][0]
        return {"score": ppl}

In [7]:
evaluator = PerplexityEvaluator()
evaluator.evaluate_strings(prediction="The rains in Spain fall mainly on the plain.")

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

{'score': 190.36891174316406}

อยากตัวอย่างใช้โมเดล GPT2 ในการหา perplexity หากจะใช้ openai gpt ไม่น่าทำได้เนื่องจากเป็นการ call API